In [ ]:
!apt-get install espeak-ng
%pip install -q git+https://github.com/phineas-pta/MatchaTTS_ngngngan.git

import IPython.display as ipd
import torch
from huggingface_hub import hf_hub_download
from matcha.cli import get_torch_device, load_matcha, load_vocoder, process_text, to_waveform

MODEL_PATH = hf_hub_download(repo_id="doof-ferb/matcha_ngngngan", filename="ckpt/checkpoint_epoch420_slim.ckpt")
VOCODER_PATH = hf_hub_download(repo_id="doof-ferb/matcha_ngngngan", filename="hifigan/g_02500000")
DEVICE = get_torch_device()
MODEL = load_matcha(MODEL_PATH, DEVICE)
VOCODER, DENOISER = load_vocoder(VOCODER_PATH, DEVICE)

@torch.inference_mode()
def tts(text, n_timesteps, temperature, length_scale, denoiser_strength):
	tmp0 = process_text(text, DEVICE)
	phones, txt, txt_len = tmp0["x_phones"][1::2], tmp0["x"], tmp0["x_lengths"]
	output = MODEL.synthesise(txt, txt_len, n_timesteps=n_timesteps, temperature=temperature, spks=None, length_scale=length_scale)
	waveform = to_waveform(output["mel"], VOCODER, DENOISER, denoiser_strength=denoiser_strength).numpy()
	ipd.display(ipd.Audio(waveform, rate=22050))
	ipd.display(ipd.Markdown("Văn bản dưới dạng mẫu tự biểu âm quốc tế (IPA), khẩu âm Hà Nội\n\n" + phones))

In [ ]:
tts(
"""Kính thưa quý vị, xin quý vị ghé vào thăm kênh Youtube Nguyễn Ngọc Ngạn,
quý vị sẽ gặp lại tất cả các băng đọc truyện của Nguyễn Ngọc Ngạn do Trung tâm Thúy Nga thực hiện
và những truyện mới cùng những buổi nói chuyện về nhiều đề tài phổ biến khác nhau.
Xin chân thành cảm ơn và chờ đón quý vị.""",
n_timesteps=50, temperature=0.667, length_scale=.95, denoiser_strength=2.5e-4
)